In [ ]:
#| default_exp models

# fastkaggle.models

> API details for fastkaggle models to help manage model weights as kaggle datasets

In [ ]:
#|hide
from nbdev.showdoc import *

In [ ]:
#|export
import os,json,subprocess, shutil
import re
from fastcore.utils import *
from fastkaggle.core import *
from fastkaggle.datasets import *
from fastkaggle.competition import *
# from fastcore.all import *

In [ ]:
#| export
def push_fastai_learner(learner, # Fastai Learner
                        model_fname, # ie `model1.pkl`
                        version_comment, # dataset versioning
                        cfg_path='.'
                ):
    '''Exports a learner and updates kaggle dataset'''
    cfg = get_config_values(cfg_path)
    
    local_path = Path(cfg_path)/cfg['data_path']/cfg['model_dataset_name']
    ds_slug = f"{cfg['datasets_username']}/{cfg['model_dataset_name']}"
    
    print(f"-----Downloading or Creating Dataset if needed")
    if local_path.exists(): pass
    elif ds_exists(ds_slug): get_dataset(ds_slug,str(local_path))
    else:                    mk_dataset(local_path,cfg['model_dataset_name'])
    
    print(local_path)
    orig_path = learner.path
    learner.path = local_path.parent
    learner.export(model_fname)
    learner.path = orig_path
    push_dataset(local_path,version_comment)

In [ ]:
from fastai.vision.all import *
import pandas as pd

path = untar_data(URLs.MNIST_SAMPLE)
df = pd.read_csv(path/'labels.csv')
dls = ImageDataLoaders.from_df(df,path)
learn = vision_learner(dls, models.resnet18, loss_func=CrossEntropyLossFlat(), ps=0.25)
push_fastai_learner(learn,'model1.pkl','testing fastkaggle')

path = Path('models-titanic')
assert path.exists()
assert ds_exists('isaacflath/models-titanic')
Path(path).ls().map(lambda x: x.unlink())
Path(path).rmdir()

-----Downloading or Creating Dataset if needed
models-titanic


## Export -

In [ ]:
#|hide
#|eval: false
from nbdev.doclinks import nbdev_export
nbdev_export()